# 📊 Factor Research – JPMorgan European Equity Thesis

This notebook performs **factor research and exposure analysis** for the JPMorgan European Equity Thesis strategy.

It is designed to work with your project structure:

- `src/analytics/factors/analyzer.py`
- `src/analytics/factors/fama_french.py`
- `src/analytics/factors/custom_factors.py`
- `src/data/connectors/yahoo.py`
- `src/utils/math_utils.py`

You can use this notebook to:

1. Load **European & US index returns**
2. Load **Fama-French & custom factor data**
3. Compute **factor exposures (betas)** for:
   - STOXX 50 / STOXX 600
   - Your JPM strategy equity curve (optional)
4. Analyze **factor contributions to returns**
5. Visualize **factor tilts vs benchmark**
6. Export factor analytics for the dashboard & reports


## 1️⃣ Environment & Imports

Run this from the **project root** where `src/` is located or let it auto-detect.

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Ensure project root is in path
PROJECT_ROOT = Path.cwd()
if (PROJECT_ROOT / 'src').exists():
    sys.path.append(str(PROJECT_ROOT))
else:
    PROJECT_ROOT = PROJECT_ROOT.parent
    sys.path.append(str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

# Project imports
from src.data.connectors.yahoo import YahooMarketData
from src.analytics.factors.analyzer import FactorAnalyzer
from src.analytics.factors.fama_french import FamaFrenchFactorModel
from src.analytics.factors.custom_factors import CustomFactorModel
from src.utils.math_utils import to_returns

plt.style.use('seaborn-v0_8')


## 2️⃣ Load Price Data (Indices & Strategy)

We'll get daily prices for:

- `^STOXX50E` – Euro STOXX 50 (or STOXX Europe 600 proxy)
- `^GSPC` – S&P 500 (global benchmark)

You can also load your **strategy equity curve** from exports.

In [ ]:
yahoo = YahooMarketData()

start_date = "2020-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")

index_tickers = ["^STOXX50E", "^GSPC"]

prices_df = yahoo.get_history_bulk(tickers=index_tickers, start=start_date, end=end_date)
prices_df = prices_df.dropna(how="all")

print("Price data shape:", prices_df.shape)
prices_df.tail()

### 2.1 Compute Daily Returns


In [ ]:
returns_df = prices_df.pct_change().dropna()
returns_df.columns = ["EU", "US"]  # rename for clarity

returns_df.describe().T

### 2.2 (Optional) Load Strategy Equity Curve

If you have exported your strategy equity curve (from `03_backtest_analysis.ipynb`) to:

- `data/exports/backtest_equity_curve.csv`

we can align that with the factor data to estimate **strategy factor exposures**.

In [ ]:
strategy_path = PROJECT_ROOT / "data" / "exports" / "backtest_equity_curve.csv"
strategy_returns = None

if strategy_path.exists():
    strat_df = pd.read_csv(strategy_path, parse_dates=["date"])
    strat_df.set_index("date", inplace=True)
    strat_eq = strat_df["strategy_equity"].dropna()
    strategy_returns = strat_eq.pct_change().dropna()
    strategy_returns.name = "Strategy"
    print("Loaded strategy equity curve and computed returns.")
else:
    print("No strategy export file found at", strategy_path)


## 3️⃣ Load Factor Data (Fama-French & Custom)

We now load factor data using your Fama-French wrapper and any custom factors you've defined.

The following factors are typically relevant:

- **MKT-RF** – Market excess return
- **SMB** – Size (Small minus Big)
- **HML** – Value (High minus Low book-to-market)
- **RMW** – Profitability
- **CMA** – Investment
- Optional regional factors (e.g., Europe-specific)


In [ ]:
ff = FamaFrenchFactorModel()

# This method name should match your implementation; adjust if needed
factor_df = ff.load_factors(start=start_date, end=end_date)
factor_df = factor_df.dropna()

print("Factor data shape:", factor_df.shape)
factor_df.head()

### 3.1 Load Custom Factors

These might include:
- **FR-DE spread**
- **Credit impulse**
- **China PMI**
- **EU valuation gap**


In [ ]:
custom_model = CustomFactorModel()

# Again, adapt method names if needed
custom_factors = custom_model.build_custom_factors(start=start_date, end=end_date)
custom_factors = custom_factors.dropna()

print("Custom factors shape:", custom_factors.shape)
custom_factors.head()

### 3.2 Align Returns & Factors


In [ ]:
# Align by date intersection
combined = returns_df.join(factor_df, how="inner").join(custom_factors, how="inner")

print("Combined dataset shape:", combined.shape)
combined.head()

## 4️⃣ Factor Exposure Estimation

We use `FactorAnalyzer` to run regressions of:

- **EU index returns** on factor returns
- **US index returns** on factor returns
- **Strategy returns (if available)** on factor returns


In [ ]:
analyzer = FactorAnalyzer()

# Define columns
asset_cols = ["EU", "US"]
factor_cols = [c for c in combined.columns if c not in asset_cols]

print("Asset columns:", asset_cols)
print("Factor columns:", factor_cols)

exposures = analyzer.compute_factor_exposures(
    asset_returns=combined[asset_cols],
    factor_returns=combined[factor_cols]
)

exposures

### 4.1 (Optional) Strategy Factor Exposures


In [ ]:
strategy_expo = None
if strategy_returns is not None:
    # Align with factor data
    aligned = pd.concat([
        strategy_returns.rename("Strategy"),
        factor_df,
        custom_factors,
    ], axis=1).dropna()

    strategy_expo = analyzer.compute_factor_exposures(
        asset_returns=aligned[["Strategy"]],
        factor_returns=aligned.drop(columns=["Strategy"])
    )
    display(strategy_expo)
else:
    print("No strategy series loaded; skipping strategy factor exposure.")


## 5️⃣ Factor Contribution to Returns

We decompose average returns into contributions from each factor.

In [ ]:
contrib = analyzer.factor_contributions(
    asset_returns=combined[asset_cols],
    factor_returns=combined[factor_cols]
)

contrib

### 5.1 Visualize Factor Contributions (Bar Chart)


In [ ]:
# Convert contributions to long format
contrib_long = contrib.reset_index().melt(id_vars="asset", var_name="factor", value_name="contribution")

fig = px.bar(
    contrib_long,
    x="factor",
    y="contribution",
    color="asset",
    barmode="group",
    title="Factor Contribution to Average Return",
)
fig.update_layout(xaxis_title="Factor", yaxis_title="Contribution (daily)")
fig.show()

## 6️⃣ Factor Tilts vs Benchmark

We compare **EU vs US** factor exposures to see where Europe is more/less exposed.

In [ ]:
# exposures is typically a DataFrame with index=asset, columns=factors
eu_expo = exposures.loc["EU"]
us_expo = exposures.loc["US"]

tilt = eu_expo - us_expo
tilt_df = pd.DataFrame({
    "factor": tilt.index,
    "tilt": tilt.values,
})

fig = px.bar(
    tilt_df,
    x="factor",
    y="tilt",
    title="Factor Tilts – Europe vs US (EU minus US)",
)
fig.update_layout(xaxis_title="Factor", yaxis_title="Exposure Difference")
fig.show()

## 7️⃣ Factor Time-Series Visualization

We can also inspect the behavior of selected factors over time.

In [ ]:
# Pick a subset of factors to visualize
selected_factors = factor_cols[:5]  # first 5 for example

fig = go.Figure()
for f in selected_factors:
    fig.add_trace(go.Scatter(
        x=combined.index,
        y=combined[f].cumsum(),
        mode="lines",
        name=f,
    ))

fig.update_layout(
    title="Cumulative Factor Returns (Sample)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    template="plotly_white",
)
fig.show()

## 8️⃣ Export Factor Analytics

We export factor exposures and contributions for use in:

- Streamlit factor view
- PDF reports
- Excel exports

In [ ]:
EXPORT_DIR = PROJECT_ROOT / "data" / "exports"
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

expo_path = EXPORT_DIR / "factor_exposures.csv"
contrib_path = EXPORT_DIR / "factor_contributions.csv"
tilt_path = EXPORT_DIR / "factor_tilts_eu_vs_us.csv"

exposures.to_csv(expo_path)
contrib.to_csv(contrib_path)
tilt_df.to_csv(tilt_path, index=False)

print("Saved factor exposures to:", expo_path)
print("Saved factor contributions to:", contrib_path)
print("Saved factor tilts to:", tilt_path)

## ✅ Summary

In this notebook we:

1. Loaded **index returns** for Europe & US
2. Loaded **Fama-French** and **custom macro factors**
3. Estimated **factor exposures** (betas) for EU, US, and optionally your strategy
4. Computed **factor contributions** to average returns
5. Measured **factor tilts** (EU vs US)
6. Exported key tables to `data/exports/` for your dashboard and reports

This gives you a **JPMorgan-grade factor research workflow** backing your European equity thesis.